In [551]:

import mysql.connector
from mysql.connector import errorcode
import logging


logging.basicConfig(level=logging.DEBUG, filename='effigySync.log', format='%(asctime)s %(name)s %(levelname)s:%(message)s')
logger = logging.getLogger(__name__)


In [552]:
import os
#SOURCEPATH='Y:/server/EFFIGY_PICS'
SOURCEPATH='Y:/server/EFFIGY_PICS2'
#SOURCEPATH = 'C:/Users/user/Pictures'


In [553]:
def fixSlashes(val):
    val = val.replace('\\', '/')
    return val    

In [554]:

a=set()
#folders = []
# r=root, d=directories, f = files
for r, d, f in os.walk(SOURCEPATH):       
    for f1 in f:
        ext = os.path.splitext(f1)[1]
        a.add(ext) 
a

{'.JPG', '.jpg'}

In [555]:
def getID(tbl,thisid, key,val):
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Ilms2009',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select " + thisid + " from " +  tbl  + " where " + key + "= %s"
        #print (ssql)
        mycursor.execute(ssql, (fixSlashes(val),))
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        newid = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            newid.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:" + val
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed"+ val)
        # obejct is a tuple we retrieve only id
        print(newid)
        if len(newid) >= 1:
            id = newid[0][0]
        else:
            id = -1
        return (id)

In [556]:
def getImageID(tbl,thisid, key,val,key2,val2):
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Ilms2009',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select " + thisid + " from " +  tbl  + " where " + key + "= %s and  " + key2 + "= %s"
        #print (ssql)
        mycursor.execute(ssql, (fixSlashes(val),val2,))
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        newid = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            newid.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:" + val
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed"+ val)
        # obejct is a tuple we retrieve only id
        print(newid)
        if len(newid) >= 1:
            id = newid[0][0]
        else:
            id = -1
        return (id)

In [557]:
newid= getImageID("media","media_id", "path","c:\\","name","test.jpg")

[]


In [577]:
def getValidExtensions():
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Ilms2009',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "select extension from supported_ext order by 1"
        mycursor.execute(ssql)
        rows = mycursor.fetchall()
        columns = [desc[0] for desc in mycursor.description]
        rec = []
        for row in rows:
            # we dont need json or column names only one row returned
            #row = dict(zip(columns, row))
            rec.append(row)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"+val
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "getID - Completed OK!:"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("getID DB Closed")
        # converting it to a SET, fast lookup
        ext_set=set()  
        for i in rec:
            ext_set.add (i[0])
        return (ext_set)

In [578]:
#get valid extensions
EXT = getValidExtensions()
#  check
'avi' in EXT
EXT

{'.avi', '.jpeg', '.jpg', '.mov', '.mp3', '.mp4', '.mpg', '.png', '.wmv'}

In [579]:
def insertRoot (root):
    logger.info("Starting Request")
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Ilms2009',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "insert into loc_root values (null, %s)"
        root = fixSlashes(root)
        #print (root)
        mycursor.execute(ssql, (root,))
        mydb.commit()
        #get newly created ID
        newid= getID("loc_root","root_id", "root",root)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "Root-Completed OK!"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("Root - DB Closed")
        return (newid)

In [580]:
getID("loc_folder","folder_id", "folder_path","C:ggg/Uffsssehhffrsggs/user/Pictures/2017-11111111111")

[]


-1

In [581]:
def insertFolder (root_id,root,folder,folderPath):
    logger.info("Starting Request")
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Ilms2009',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "insert into loc_folder values (null,%s, %s,%s,%s)"
        root = fixSlashes(root)
        folder = fixSlashes(folder)
        folderPath = fixSlashes(folderPath)
        #print (ssql)
        mycursor.execute(ssql, (root_id,root,folder,folderPath,))
        mydb.commit()
        #get newly created ID
        newid= getID("loc_folder","folder_id", "folder_path",folderPath)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "Folder Completed OK!"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("Folder check DB Closed")
        return (newid)

In [582]:
def skipRoot(root):
    skip=False
    toSkip="Y:\server\EFFIGY_PICS\Zdjecia\\tomcat-ZDJECIA"
    #print("toskip",fixSlashes(toSkip).upper())
    #print("root  ",fixSlashes(root).upper())
    len(root)
    len(toSkip)
    if len(root) >= len(toSkip):
        if (fixSlashes(toSkip).upper() == fixSlashes(root[:len(toSkip)]).upper())  :
            skip=True
    #else:
    #    print("not matching length")
    return(skip)

In [583]:
skipRoot("Y:\server\EFFIGY_PICS\Zdjecia\\tomcat-ZdJECIA")

True

In [584]:
#newid = insertFolder(1, "C:/Users/user/Pictures","2017-1111", "C:ggg/Uffsssehhffrsggs/user/Pictures/2017-1221111111111")
#newid

In [585]:
def inserImage (file_name,media_type,folder_id,path, parent_folder_id,parent_folder,attrib_id,ext):
    logger.info("Saving image")
    print(file_name,media_type,folder_id,path, parent_folder_id,parent_folder,attrib_id,ext)
    try:
        mydb = mysql.connector.connect(user='kogutc', password='Ilms2009',host='127.0.0.1',database='effigy',auth_plugin='mysql_native_password')
        mycursor = mydb.cursor()
        ssql = "insert into media values (null,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        path = fixSlashes(path)
        parent_folder = fixSlashes(parent_folder)
        #print (ssql)
        mycursor.execute(ssql, ( file_name,media_type,folder_id,path,parent_folder_id,parent_folder,"sDesc",file_name,0,ext,'NULL','NULL'))
        mydb.commit()
        #get newly created ID
        newid= getImageID("media","media_id", "path",path,"name",file_name)
    except Exception as err:
        logger.error(err)
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            custError = "Something is wrong with your user name or password"
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            custError = "Database does not exists"
        else:
            custError = err
        print(custError)
        logger.error(custError)
        mycursor.close()
        mydb.close()
    else:
        custError = "Folder Completed OK!"
        logger.info(custError)
        mycursor.close()
        mydb.close()
        logger.info("Folder check DB Closed")
        return (newid)

In [588]:
def importFolders():
    #get Valid extensions first
    c=0
    EXT = getValidExtensions()
    #  check
    '' in EXT
    print(EXT)
    for r, d, f in os.walk(SOURCEPATH):
        #print("dir",d,r)
        if len(d) >0:
            if  skipRoot(r):
                print("Skipped Root:",r)
            else:
                idR = insertRoot(r)
                #print("Processed Root:",idR,r)
                for dir in d:
                    print ("will INSERT dir:",idR,dir)
                    #print("about to insert", idR,r,dir,r + '/' + dir)
                    idF = insertFolder(idR,r,dir,r + "/"+ dir)
                    #print("Created folder",idF)
                    pth = r +"/"+dir
                    print("path", pth)
                    files = [f for f in os.listdir(pth) if os.path.isfile(os.path.join(pth,f))]
                    for f in files:
                        file_ext= os.path.splitext(f)[1].lower()
                        print(file_ext,ext)
                        if (file_ext in EXT):
                            print("Processing:" + f)
                            #inserImage(file_name,media_type,folder_id,path, parent_folder_id,parent_folder,attrib_id,ext):                       
                            idM = inserImage(f,'P',idF, r +"/"+dir, idR,r,"",file_ext)
                            print(idM)
                           
    return(c)                    
                    
    
                    
                    
                
        
    

In [589]:
c= importFolders()
print ("total:",c)

{'.mp4', '.jpg', '.mov', '.mpg', '.png', '.jpeg', '.avi', '.mp3', '.wmv'}
[(1,)]
will INSERT dir: 1 Zdjecia
[(1,)]
path Y:/server/EFFIGY_PICS2/Zdjecia
.jpg .jpg
Processing:20041008 001.JPG
20041008 001.JPG P 1 Y:/server/EFFIGY_PICS2/Zdjecia 1 Y:/server/EFFIGY_PICS2  .jpg
[(1,)]
1
.jpg .jpg
Processing:20041008 002.jpg
20041008 002.jpg P 1 Y:/server/EFFIGY_PICS2/Zdjecia 1 Y:/server/EFFIGY_PICS2  .jpg
[(2,)]
2
[(2,)]
will INSERT dir: 2 other
[(2,)]
path Y:/server/EFFIGY_PICS2\Zdjecia/other
.jpg .jpg
Processing:IMG_0025.jpg
IMG_0025.jpg P 2 Y:/server/EFFIGY_PICS2\Zdjecia/other 2 Y:/server/EFFIGY_PICS2\Zdjecia  .jpg
[(3,)]
3
.jpg .jpg
Processing:IMG_0028.jpg
IMG_0028.jpg P 2 Y:/server/EFFIGY_PICS2\Zdjecia/other 2 Y:/server/EFFIGY_PICS2\Zdjecia  .jpg
[(4,)]
4
.jpg .jpg
Processing:IMG_0131.jpg
IMG_0131.jpg P 2 Y:/server/EFFIGY_PICS2\Zdjecia/other 2 Y:/server/EFFIGY_PICS2\Zdjecia  .jpg
[(5,)]
5
.jpg .jpg
Processing:IMG_0132.jpg
IMG_0132.jpg P 2 Y:/server/EFFIGY_PICS2\Zdjecia/other 2 Y:/server/

In [516]:
id = getID("loc_root","root_id","root","C:/Users/user/Pictures/2018-06")
id

[]


-1

In [517]:
c


0

In [518]:
id = getID("loc_folder","folder_id","folder_path","C:/Users/user/Pictures/2018-06")
id


[]


-1

In [ ]:
getSampleData()

In [ ]:
path = "C:/GIT_REPOSITORY/REPO/effigy/imageLib/test.jpg"
from PIL import Image
# Read image 
img = Image.open(path) 
  
# Output Images 
img.show() 

# prints format of image 
print(img.format) 

# prints mode of image 
print(img.mode) 

In [ ]:
for r, d, f in os.walk(path):
    print ("directory is: ", d)
    

In [ ]:
for f in files:
    print(f)
    

In [ ]:
for r, d, f in os.walk(path):
    print(f)

In [ ]:
from PIL import Image
import glob, os
size = 128, 128
for infile in glob.glob(path):
    file, ext = os.path.splitext(infile)
    im = Image.open(infile)
    im.thumbnail(size)
    im.save(file + ".thumbnail1.png", "PNG")

In [ ]:

path = "C:/GIT_REPOSITORY/REPO/effigy/imageLib/geotag.jpg"

# Python program to read 
# image using PIL module 
  
# importing PIL 
from PIL import Image 
  
# Read image 
img = Image.open(path) 
  
# Output Images 
#img.show() 
  
# prints format of image 
print(img.format) 
  
    
    

# prints mode of image 
print(img.mode) 
dic = img._getexif()
width = dic.get(40963)
height = dic.get(40962)
camera = dic.get(271)
model = dic.get(272)
dttaken = dic.get(36868)
dtmod= dic.get(36867)
fstop= dic.get(42036)
exposure = dic.get(40960) 
orientation=dic.get(274)
flash = dic.get(7385)
shutter = dic.get(37377)
apperture=dic.get(37378) 
isospeed=dic.get(34855) 
digzoom=dic.get(41988)
exif=""


In [ ]:
dttaken

In [ ]:
exif=""
for x in dic:
     if x != 37500:
        exif= exif + (str(x) +  ":" +  str(dic.get(x)) + chr(13) +chr(10) )  
exif

In [ ]:
for x in dic:
    if x != 37500:
     print(x,":", dic.get(x))

In [ ]:
import os
path = 'C:/Users/user/Pictures'
roots = []
# r=root, d=directories, f = files
for r,d,f in os.walk(path):
    print (r,d)
    for f in d:
       print(f)
   